# ROADMAP (TODO -> Apagar depois)

1. Definir a diferenca metodologica entre as duas fontes
2. Tornar os dados comparaveis
3. Quantificar a diferenca entre os dois sistemas (Tom e Gov) em horas e percentual
4. Padroes ou distorcoes por escola, funcao ou mes

Passos
1. Cruzar dados -> Tom e Base Gov (entender as métricas de carga horária)
    1.1 Diferença entre regencia e nao regencia
    1.2 Demissão (como avaliar)
    1.3 Entender quantas horas, cada professor, por mes, trabalhou (cruzando os dados entre estimativa Tom e 
    estimativa GOV)
    1.4 Identificar discrepencias (horas faltantes)
    
2. Calcular pagamento por mês a cada professor baseado no cáculo de regencia e não regencia e por escola
3. Identificar quantidade de horas totais de professores em cada escola por tipo de funcao, valor, e CH


# **O desafio**

Atualmente, a Tom atua na gestão da educação pública em parceria com o gorverno. O desafio parte da necessidade da Tom identificar e analisar as principais diferenças entre um de seus serviços, a contratação e gestão de carga horária de professores. 

In [181]:
# Manipulação dos dados
import pandas as pd
import numpy as np

# Visualização dos dados
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

from warnings import filterwarnings

filterwarnings('ignore')

# Manipulacao de arquivos e caminhos
from pathlib import Path

ROOT_DIR = Path().cwd().parent
DATA_DIR = ROOT_DIR / "data"
FILE_PATH = DATA_DIR / "data.xlsx"

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

sns.set_theme(
    context="notebook",
    style="whitegrid",
    palette="colorblind",
    font="Arial",
    font_scale=1.0
)
plt.rc("figure", figsize=(12, 6), dpi=120)


In [182]:
sheets = pd.ExcelFile(FILE_PATH)

names = sheets.sheet_names

names.remove('Auxiliares>>>>')

In [183]:
SHEET_KEYS = {
    i: name
    for i, name in enumerate(names)
}

SHEET_KEYS

{0: 'Resumo Executivo',
 1: 'Análise CH por Escola',
 2: 'BD_Tom Folha',
 3: 'BD_SistemaGov',
 4: 'Valores por escola',
 5: 'Página30'}

In [184]:
hc_per_school = pd.read_excel(FILE_PATH, sheet_name=SHEET_KEYS[1])

print(hc_per_school.shape)
hc_per_school.head()

(128, 9)


,Mês,Escola,CÓDIGO ESCOLA,Valor Unitário Carga Horária,CH Regência,CH Sem Regência,Valor Regência,Valor Sem Regência,Valor CH Mensal
0,janeiro,ESCOLA A,ABC1,NaN,NaN,NaN,NaN,NaN,NaN
1,janeiro,ESCOLA B,ABC2,NaN,NaN,NaN,NaN,NaN,NaN
2,janeiro,ESCOLA C,ABC3,NaN,NaN,NaN,NaN,NaN,NaN
3,janeiro,ESCOLA D,ABC4,NaN,NaN,NaN,NaN,NaN,NaN
4,janeiro,ESCOLA E,ABC5,NaN,NaN,NaN,NaN,NaN,NaN


In [185]:
tom_payroll = pd.read_excel(FILE_PATH, sheet_name=SHEET_KEYS[2])

print(tom_payroll.shape)
tom_payroll.head()

(3085, 12)


,Mês,Escola,CodEscola,Nome,CodFuncionario,SITUACAO,ADM,DESCRICAO_CARGO,DESCRICAO_VERBA,Horas Mes,Categoria,Dt demissão
0,Fevereiro,ESCOLA A,ABC1,Andressa Aguiar,100.0,Trabalhando,2025-02-03,COORDENADOR(A) DE CURSO,HORAS,70.0,NAO REGENCIA,NaT
1,Fevereiro,ESCOLA A,ABC1,Andressa Aguiar,100.0,Trabalhando,2025-02-03,PROFESSOR(A),HORAS,45.0,REGENCIA,NaT
2,Fevereiro,ESCOLA A,ABC1,Márcio Wagner,101.0,Trabalhando,2025-02-03,PROFESSOR(A),HORAS,27.0,REGENCIA,NaT
3,Fevereiro,ESCOLA A,ABC1,Igor Souza Quirino,102.0,Trabalhando,2025-02-03,PROFESSOR(A),HORAS,49.5,REGENCIA,NaT
4,Fevereiro,ESCOLA A,ABC1,João Miguel Oliveira Vieira,103.0,Trabalhando,2025-02-03,PROFESSOR(A),HORAS,40.5,REGENCIA,NaT


In [186]:
gov_payroll = pd.read_excel(FILE_PATH, sheet_name=SHEET_KEYS[3])

print(gov_payroll.shape)
gov_payroll.head()

(6780, 14)


,Mês,Escola,CodEscola,Nome Funcionario,CodFuncionario,Tipo,DisciplinaFuncao,DisciplinaFuncaoDescricao,PeriodoIni,PeriodoFim,DiasTrabalhados,CHSuprimento,CHDiaria,CHPeriodo
0,Fevereiro,ESCOLA Z,ABC23,Bruna Diniz,680,REGENCIA,6279,LITER ART E MOVIMENTO I,2025-02-03,2025-02-28,26,8,1.142857,29.714282
1,Fevereiro,ESCOLA Z,ABC23,Bruna Diniz,680,REGENCIA,6363,ELETIVA I,2025-02-03,2025-02-28,26,4,0.571428,14.857128
2,Fevereiro,ESCOLA Z,ABC23,Yolanda Sales,681,REGENCIA,704,ARTE,2025-02-03,2025-02-28,26,8,1.142857,29.714282
3,Fevereiro,ESCOLA Z,ABC23,Yolanda Sales,681,REGENCIA,732,ARTE II,2025-02-03,2025-02-28,26,2,0.285714,7.428564
4,Fevereiro,ESCOLA Z,ABC23,Yolanda Sales,681,REGENCIA,5641,PROJETO DE VIDA I,2025-02-03,2025-02-28,26,25,3.571428,92.857128


Aqui possuímos a primeira discrespância entre os dados do governo. A CH total do funcionários estão sendo calculadas sem levar em conta as horas suplementares, isto é, horas-extras não estão sendo contabilizadas ao fazer a auditoria dos empregados. Após verificar se temos valores nulos, iremos corrigir a CHPeriodo para que ela esteja de acordo com as horas de cada funcionario

In [187]:
school_values = pd.read_excel(FILE_PATH, sheet_name=SHEET_KEYS[4])

print(school_values.shape)
school_values.head()

(32, 3)


,Escola,CodEscola,Valor Unitário Carga Horária
0,ESCOLA Z,ABC23,23.09
1,ESCOLA N,ABC14,29.27
2,ESCOLA O,ABC15,29.27
3,ESCOLA E,ABC5,29.27
4,ESCOLA F,ABC6,27.21


In [188]:
teachers_match = pd.read_excel(FILE_PATH, sheet_name=SHEET_KEYS[5])

teachers_match.head()

,UNIQUE,Unnamed: 1,Unnamed: 2,conit,Unnamed: 4,UNIQUE.1,CÓDIGO ESCOLA,Escola
0,61008303100,100,Andressa Aguiar,1,NaN,41128656.0,ABC1,ESCOLA A
1,9966162925,101,Márcio Wagner,1,NaN,41129792.0,ABC2,ESCOLA B
2,638731925,102,Igor Souza Quirino,1,NaN,41131479.0,ABC3,ESCOLA C
3,5743855927,103,João Miguel Oliveira Vieira,1,NaN,41131908.0,ABC4,ESCOLA D
4,2334064920,104,Dennis Oshima,1,NaN,41135784.0,ABC5,ESCOLA E


### Limpeza das tabelas

In [189]:
print(hc_per_school[['Mês', 'Escola']].info())

grouped_schools = hc_per_school.groupby('Mês')['Escola'].count()

grouped_schools

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Mês     128 non-null    object
 1   Escola  128 non-null    object
dtypes: object(2)
memory usage: 2.1+ KB
None


Mês
abril        32
fevereiro    32
janeiro      32
março        32
Name: Escola, dtype: int64

In [190]:
print(tom_payroll.info())
print(tom_payroll.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3085 entries, 0 to 3084
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Mês              3085 non-null   object        
 1   Escola           3085 non-null   object        
 2   CodEscola        3085 non-null   object        
 3   Nome             3078 non-null   object        
 4   CodFuncionario   3078 non-null   float64       
 5   SITUACAO         3085 non-null   object        
 6   ADM              1964 non-null   datetime64[ns]
 7   DESCRICAO_CARGO  3085 non-null   object        
 8   DESCRICAO_VERBA  3085 non-null   object        
 9   Horas Mes        3085 non-null   float64       
 10  Categoria        3085 non-null   object        
 11  Dt demissão      189 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(2), object(8)
memory usage: 289.3+ KB
None
       CodFuncionario                            ADM    Horas Mes  \
coun

In [191]:
tom_payroll.isna().sum()

Mês                   0
Escola                0
CodEscola             0
Nome                  7
CodFuncionario        7
SITUACAO              0
ADM                1121
DESCRICAO_CARGO       0
DESCRICAO_VERBA       0
Horas Mes             0
Categoria             0
Dt demissão        2896
dtype: int64

In [192]:
tom_payroll[tom_payroll['Nome'].isna()]

,Mês,Escola,CodEscola,Nome,CodFuncionario,SITUACAO,ADM,DESCRICAO_CARGO,DESCRICAO_VERBA,Horas Mes,Categoria,Dt demissão
1994,Abril,ESCOLA A,ABC1,NaN,NaN,Trabalhando,NaT,PROFESSOR(A),HORAS,54.0,REGENCIA,NaT
2047,Abril,ESCOLA G,ABC7,NaN,NaN,Trabalhando,NaT,PROFESSOR(A),HORAS,18.9,REGENCIA,NaT
2640,Abril,ESCOLA W,ABC21,NaN,NaN,Trabalhando,NaT,PROFESSOR(A),HORAS,18.0,REGENCIA,NaT
2641,Abril,ESCOLA W,ABC21,NaN,NaN,Trabalhando,NaT,PROFESSOR(A),HORAS,20.0,REGENCIA,NaT
2800,Abril,ESCOLA Z,ABC23,NaN,NaN,Trabalhando,NaT,COORDENADOR(A) PEDAGOGICO,HORAS,100.0,NAO REGENCIA,NaT
2859,Abril,ESCOLA J,ABC10,NaN,NaN,Trabalhando,NaT,PROFESSOR(A),HORAS,14.7,REGENCIA,NaT
3048,Abril,ESCOLA M,ABC13,NaN,NaN,Trabalhando,NaT,PROFESSOR(A),HORAS,12.6,REGENCIA,NaT


In [193]:
tom_payroll[tom_payroll['CodFuncionario'].isna()]

,Mês,Escola,CodEscola,Nome,CodFuncionario,SITUACAO,ADM,DESCRICAO_CARGO,DESCRICAO_VERBA,Horas Mes,Categoria,Dt demissão
1994,Abril,ESCOLA A,ABC1,NaN,NaN,Trabalhando,NaT,PROFESSOR(A),HORAS,54.0,REGENCIA,NaT
2047,Abril,ESCOLA G,ABC7,NaN,NaN,Trabalhando,NaT,PROFESSOR(A),HORAS,18.9,REGENCIA,NaT
2640,Abril,ESCOLA W,ABC21,NaN,NaN,Trabalhando,NaT,PROFESSOR(A),HORAS,18.0,REGENCIA,NaT
2641,Abril,ESCOLA W,ABC21,NaN,NaN,Trabalhando,NaT,PROFESSOR(A),HORAS,20.0,REGENCIA,NaT
2800,Abril,ESCOLA Z,ABC23,NaN,NaN,Trabalhando,NaT,COORDENADOR(A) PEDAGOGICO,HORAS,100.0,NAO REGENCIA,NaT
2859,Abril,ESCOLA J,ABC10,NaN,NaN,Trabalhando,NaT,PROFESSOR(A),HORAS,14.7,REGENCIA,NaT
3048,Abril,ESCOLA M,ABC13,NaN,NaN,Trabalhando,NaT,PROFESSOR(A),HORAS,12.6,REGENCIA,NaT


In [194]:
print(gov_payroll.isna().sum())
print(gov_payroll.info())
print(gov_payroll.describe())

Mês                          0
Escola                       0
CodEscola                    0
Nome Funcionario             0
CodFuncionario               0
Tipo                         0
DisciplinaFuncao             0
DisciplinaFuncaoDescricao    0
PeriodoIni                   0
PeriodoFim                   0
DiasTrabalhados              0
CHSuprimento                 0
CHDiaria                     0
CHPeriodo                    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6780 entries, 0 to 6779
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Mês                        6780 non-null   object        
 1   Escola                     6780 non-null   object        
 2   CodEscola                  6780 non-null   object        
 3   Nome Funcionario           6780 non-null   object        
 4   CodFuncionario             6780 non-null   int64         
 5   

Ao analisar a tabela do Governo, notamos que o cálculo de horas totais CHPeriodo, nao leva em conta horas extras (CHSuprimento). Dessa forma, realizaremos um novo calculo para o periodo total trabalhado em uma nova coluna seguindo a seguinte formula:

CHPeriodo_new = DiasTrabalhados * CHDiaria + CHSuprimento

In [195]:
gov_payroll['CHPeriodo_new'] = gov_payroll['DiasTrabalhados'] * gov_payroll['CHDiaria'] + gov_payroll['CHSuprimento']

hours_diff_total = gov_payroll['CHPeriodo_new'].sum() - gov_payroll['CHPeriodo'].sum()
hours_diff_mean = (gov_payroll['CHPeriodo_new'] - gov_payroll['CHPeriodo']).mean()

print(f'Foram computadas erroneamente um total de {hours_diff_total:.2f} horas A MENOS dos funcionários, com uma média de {hours_diff_mean:.2f} horas faltantes')

Foram computadas erroneamente um total de 58716.00 horas A MENOS dos funcionários, com uma média de 8.66 horas faltantes


Ao realizar uma inspeção das tabelas nota-se que a folha de pagamento da Tom possui valores faltantes. Dessa forma, é essencial para análise descobrir a origem desse erro (duplicatas, erros de inserção e etc.). Além disso, todos os valores faltantes se referem ao mês de abril.

In [196]:
april_tom =  tom_payroll[tom_payroll['Mês'] == 'Abril']

missing_april_tom = april_tom[(april_tom['CodFuncionario'].isna()) & (april_tom['Nome'].isna())]

missing_april_tom

,Mês,Escola,CodEscola,Nome,CodFuncionario,SITUACAO,ADM,DESCRICAO_CARGO,DESCRICAO_VERBA,Horas Mes,Categoria,Dt demissão
1994,Abril,ESCOLA A,ABC1,NaN,NaN,Trabalhando,NaT,PROFESSOR(A),HORAS,54.0,REGENCIA,NaT
2047,Abril,ESCOLA G,ABC7,NaN,NaN,Trabalhando,NaT,PROFESSOR(A),HORAS,18.9,REGENCIA,NaT
2640,Abril,ESCOLA W,ABC21,NaN,NaN,Trabalhando,NaT,PROFESSOR(A),HORAS,18.0,REGENCIA,NaT
2641,Abril,ESCOLA W,ABC21,NaN,NaN,Trabalhando,NaT,PROFESSOR(A),HORAS,20.0,REGENCIA,NaT
2800,Abril,ESCOLA Z,ABC23,NaN,NaN,Trabalhando,NaT,COORDENADOR(A) PEDAGOGICO,HORAS,100.0,NAO REGENCIA,NaT
2859,Abril,ESCOLA J,ABC10,NaN,NaN,Trabalhando,NaT,PROFESSOR(A),HORAS,14.7,REGENCIA,NaT
3048,Abril,ESCOLA M,ABC13,NaN,NaN,Trabalhando,NaT,PROFESSOR(A),HORAS,12.6,REGENCIA,NaT


Para descobrirmos quais valores faltam (ou se são apenas erros) precisamos entender como o cálculo de carga horária de Tom é feito, visto que entendemos como a auditoria governamental é feita. As CH de Tom é calculada da seguinte forma:

1. Extrai-se a quantidade CHDiaria de cada funcionaria agregada a sua funcao no mes (Regencia ou Nao Regencia)
2. As horas diarias são multiplicadas por 7 (CH Semanal)
3. Esse CH Semanal é então multiplicado pelo seu fato (relativo a funcao)

Dessa forma, podemos decompor a coluna Hora Mes que representa a CH Mensal total de Tom em CH Semanal e entender qual a origem dos valores com N/A.

In [197]:
tom_copy = tom_payroll.copy()

tom_copy['fator'] = tom_payroll['Categoria']\
    .map({'REGENCIA': 4.5, 'NAO REGENCIA': 5})

tom_copy['CH_Semanal'] = tom_copy['Horas Mes'] / tom_copy['fator']

tom_copy.head()

,Mês,Escola,CodEscola,Nome,CodFuncionario,SITUACAO,ADM,DESCRICAO_CARGO,DESCRICAO_VERBA,Horas Mes,Categoria,Dt demissão,fator,CH_Semanal
0,Fevereiro,ESCOLA A,ABC1,Andressa Aguiar,100.0,Trabalhando,2025-02-03,COORDENADOR(A) DE CURSO,HORAS,70.0,NAO REGENCIA,NaT,5.0,14.0
1,Fevereiro,ESCOLA A,ABC1,Andressa Aguiar,100.0,Trabalhando,2025-02-03,PROFESSOR(A),HORAS,45.0,REGENCIA,NaT,4.5,10.0
2,Fevereiro,ESCOLA A,ABC1,Márcio Wagner,101.0,Trabalhando,2025-02-03,PROFESSOR(A),HORAS,27.0,REGENCIA,NaT,4.5,6.0
3,Fevereiro,ESCOLA A,ABC1,Igor Souza Quirino,102.0,Trabalhando,2025-02-03,PROFESSOR(A),HORAS,49.5,REGENCIA,NaT,4.5,11.0
4,Fevereiro,ESCOLA A,ABC1,João Miguel Oliveira Vieira,103.0,Trabalhando,2025-02-03,PROFESSOR(A),HORAS,40.5,REGENCIA,NaT,4.5,9.0


In [198]:
april_gov_filtered = gov_payroll[gov_payroll['Mês'] == 'Abril']
missing_school_codes = missing_april_tom['CodEscola'].values

april_gov_filtered['CHSemanal'] = np.round(april_gov_filtered['CHDiaria'] * 7, 2)

april_gov_filtered.head()

,Mês,Escola,CodEscola,Nome Funcionario,CodFuncionario,Tipo,DisciplinaFuncao,DisciplinaFuncaoDescricao,PeriodoIni,PeriodoFim,DiasTrabalhados,CHSuprimento,CHDiaria,CHPeriodo,CHPeriodo_new,CHSemanal
4287,Abril,ESCOLA Z,ABC23,Bruna Diniz,680,REGENCIA,6279,LITER ART E MOVIMENTO I,2025-04-01,2025-04-30,30,8,1.142857,34.28571,42.28571,8.0
4288,Abril,ESCOLA Z,ABC23,Bruna Diniz,680,REGENCIA,6363,ELETIVA I,2025-04-01,2025-04-30,30,4,0.571428,17.14284,21.14284,4.0
4289,Abril,ESCOLA Z,ABC23,Yolanda Sales,681,REGENCIA,704,ARTE,2025-04-01,2025-04-30,30,8,1.142857,34.28571,42.28571,8.0
4290,Abril,ESCOLA Z,ABC23,Yolanda Sales,681,REGENCIA,732,ARTE II,2025-04-01,2025-04-30,30,2,0.285714,8.57142,10.57142,2.0
4291,Abril,ESCOLA Z,ABC23,Tainara Klein,682,REGENCIA,106,LINGUA PORTUGUESA,2025-04-01,2025-04-30,30,4,0.571428,17.14284,21.14284,4.0


Observamos também, que apenas 6 escolas possuem esses valores que faltam. Dessa forma, podemos investigar uma a uma afim de encontrar quais registros batem.

In [199]:
april_gov_per_missing_schools = april_gov_filtered[april_gov_filtered['CodEscola'].isin(missing_school_codes)]
april_gov_per_missing_schools.head()

,Mês,Escola,CodEscola,Nome Funcionario,CodFuncionario,Tipo,DisciplinaFuncao,DisciplinaFuncaoDescricao,PeriodoIni,PeriodoFim,DiasTrabalhados,CHSuprimento,CHDiaria,CHPeriodo,CHPeriodo_new,CHSemanal
4287,Abril,ESCOLA Z,ABC23,Bruna Diniz,680,REGENCIA,6279,LITER ART E MOVIMENTO I,2025-04-01,2025-04-30,30,8,1.142857,34.28571,42.28571,8.0
4288,Abril,ESCOLA Z,ABC23,Bruna Diniz,680,REGENCIA,6363,ELETIVA I,2025-04-01,2025-04-30,30,4,0.571428,17.14284,21.14284,4.0
4289,Abril,ESCOLA Z,ABC23,Yolanda Sales,681,REGENCIA,704,ARTE,2025-04-01,2025-04-30,30,8,1.142857,34.28571,42.28571,8.0
4290,Abril,ESCOLA Z,ABC23,Yolanda Sales,681,REGENCIA,732,ARTE II,2025-04-01,2025-04-30,30,2,0.285714,8.57142,10.57142,2.0
4291,Abril,ESCOLA Z,ABC23,Tainara Klein,682,REGENCIA,106,LINGUA PORTUGUESA,2025-04-01,2025-04-30,30,4,0.571428,17.14284,21.14284,4.0


In [200]:
april_gov_per_missing_schools_1 = april_gov_per_missing_schools[(april_gov_filtered['CodEscola'] == missing_school_codes[0]) & (april_gov_per_missing_schools['Tipo'] == 'REGENCIA')]

april_gov_per_missing_schools_1.head()

,Mês,Escola,CodEscola,Nome Funcionario,CodFuncionario,Tipo,DisciplinaFuncao,DisciplinaFuncaoDescricao,PeriodoIni,PeriodoFim,DiasTrabalhados,CHSuprimento,CHDiaria,CHPeriodo,CHPeriodo_new,CHSemanal
4765,Abril,ESCOLA A,ABC1,Andressa Aguiar,100,REGENCIA,4024,NOCOES DE DIREITO,2025-04-01,2025-04-30,30,2,0.285714,8.57142,10.57142,2.0
4766,Abril,ESCOLA A,ABC1,Andressa Aguiar,100,REGENCIA,4129,PRINCIPIOS DE ADMINISTRACAO,2025-04-01,2025-04-30,30,2,0.285714,8.57142,10.57142,2.0
4767,Abril,ESCOLA A,ABC1,Andressa Aguiar,100,REGENCIA,4450,RECURSOS HUMANOS,2025-04-01,2025-04-30,30,4,0.571428,17.14284,21.14284,4.0
4768,Abril,ESCOLA A,ABC1,Andressa Aguiar,100,REGENCIA,4767,TECNOLOGIAS E FERRAM DE GESTAO,2025-04-01,2025-04-30,30,2,0.285714,8.57142,10.57142,2.0
4769,Abril,ESCOLA A,ABC1,Carla Leal,876,REGENCIA,367,REDACAO E LEITURA,2025-04-01,2025-04-30,30,12,1.714285,51.42855,63.42855,12.0


In [201]:
april_gov_agg_1 = april_gov_per_missing_schools_1[['Nome Funcionario', 'Tipo', 'CHPeriodo', 'CHPeriodo_new', 'CHSemanal', 'CodFuncionario']].groupby(['Nome Funcionario', 'Tipo', 'CodFuncionario'])[['CHPeriodo', 'CHPeriodo_new', 'CHSemanal']].sum().reset_index()


april_gov_agg_1.sort_values('CHSemanal')

,Nome Funcionario,Tipo,CodFuncionario,CHPeriodo,CHPeriodo_new,CHSemanal
21,Ursula Rios,REGENCIA,877,12.857130,15.857130,3.0
17,Márcio Wagner,REGENCIA,101,25.714260,31.714260,6.0
22,Xandra Mota,REGENCIA,175,25.714260,31.714260,6.0
3,Carlos Zago,REGENCIA,113,28.285686,36.285686,8.0
18,Nathália Quintão,REGENCIA,924,23.999997,31.999997,8.0
12,Joana Goulart,REGENCIA,126,34.285680,42.285680,8.0
13,João Miguel Oliveira Vieira,REGENCIA,103,38.571390,47.571390,9.0
20,Ramon Jorge,REGENCIA,108,1.285713,10.285713,9.0
9,Gilberto Ferreira Oshima,REGENCIA,122,42.857100,52.857100,10.0
0,Andressa Aguiar,REGENCIA,100,42.857100,52.857100,10.0


In [202]:
print(len(april_gov_agg_1))

24


Feito a agregacao de horas semanais pela primeira escola, podemos verificar quais professores nao aparecem em Tom

In [203]:
april_tom_regency_1 = april_tom[(april_tom['Categoria'] == 'REGENCIA') & (april_tom['CodEscola'] == missing_school_codes[0])]

april_tom_regency_1['fator'] = april_tom_regency_1['Categoria']\
    .map({'REGENCIA': 4.5, 'NAO REGENCIA': 5})

april_tom_regency_1['CH_Semanal'] = np.round(april_tom_regency_1['Horas Mes'] / april_tom_regency_1['fator'], 2)

april_tom_regency_1.sort_values('CH_Semanal')

,Mês,Escola,CodEscola,Nome,CodFuncionario,SITUACAO,ADM,DESCRICAO_CARGO,DESCRICAO_VERBA,Horas Mes,Categoria,Dt demissão,fator,CH_Semanal
1974,Abril,ESCOLA A,ABC1,Ramon Jorge,108.0,Demitido,NaT,PROFESSOR(A),HORAS,13.50,REGENCIA,2025-04-10,4.5,3.00
1973,Abril,ESCOLA A,ABC1,Ursula Rios,877.0,Trabalhando,NaT,PROFESSOR(A),HORAS,13.50,REGENCIA,NaT,4.5,3.00
1967,Abril,ESCOLA A,ABC1,Márcio Wagner,101.0,Trabalhando,NaT,PROFESSOR(A),HORAS,27.00,REGENCIA,NaT,4.5,6.00
1988,Abril,ESCOLA A,ABC1,Xandra Mota,175.0,Trabalhando,NaT,PROFESSOR(A),HORAS,27.00,REGENCIA,NaT,4.5,6.00
1978,Abril,ESCOLA A,ABC1,Carlos Zago,113.0,Trabalhando,NaT,PROFESSOR(A),HORAS,29.25,REGENCIA,NaT,4.5,6.50
1965,Abril,ESCOLA A,ABC1,Andressa Aguiar,100.0,Trabalhando,NaT,PROFESSOR(A),HORAS,45.00,REGENCIA,NaT,4.5,10.00
1987,Abril,ESCOLA A,ABC1,Penélope Siqueira,120.0,Trabalhando,NaT,PROFESSOR(A),HORAS,45.00,REGENCIA,NaT,4.5,10.00
1990,Abril,ESCOLA A,ABC1,Gilberto Ferreira Oshima,122.0,Trabalhando,NaT,PROFESSOR(A),HORAS,45.00,REGENCIA,NaT,4.5,10.00
1995,Abril,ESCOLA A,ABC1,Joana Goulart,126.0,Trabalhando,NaT,PROFESSOR(A),HORAS,45.00,REGENCIA,NaT,4.5,10.00
1968,Abril,ESCOLA A,ABC1,Igor Souza Quirino,102.0,Trabalhando,NaT,PROFESSOR(A),HORAS,49.50,REGENCIA,NaT,4.5,11.00


In [204]:
print(len(april_tom_regency_1))

25


Ok, descobrimos algo interessante. Na base do Governo, para o mes de abril possuímos 24 registros de professores na Escola ABC 1. Entretanto, em Tom possuimos 25 (a instancia com valor nulo), isso revela que, possivelmente esse registro se deve a uma duplicata ou insercao errada. Portanto, vamos checar como estao os outros registros nulos das outras escolas, porém este é um forte indicativo de que estes dados se tratam de insercoes erroneas. Além disso, vamos resgatar os indices dos valores inconsistentes.

In [205]:
missings_values_idxs = []

missings_values_idxs.append(april_tom_regency_1[april_tom_regency_1['CodFuncionario'].isna()].index)

missings_values_idxs

[Index([1994], dtype='int64')]

In [206]:
april_gov_per_missing_schools_2 = april_gov_per_missing_schools[(april_gov_filtered['CodEscola'] == missing_school_codes[1]) & (april_gov_per_missing_schools['Tipo'] == 'REGENCIA')]

april_gov_agg_2 = april_gov_per_missing_schools_2[['Nome Funcionario', 'Tipo', 'CHPeriodo', 'CHPeriodo_new', 'CHSemanal', 'CodFuncionario']].groupby(['Nome Funcionario', 'Tipo', 'CodFuncionario'])[['CHPeriodo', 'CHPeriodo_new', 'CHSemanal']].sum().reset_index()


april_gov_agg_2.sort_values('CHSemanal')

,Nome Funcionario,Tipo,CodFuncionario,CHPeriodo,CHPeriodo_new,CHSemanal
11,Kauê Serra,REGENCIA,138,17.142840,21.142840,4.0
8,Isadora Donato,REGENCIA,165,34.285710,42.285710,8.0
15,Norberto Souza Zanetti,REGENCIA,159,42.857130,52.857130,10.0
2,Alícia Nakano,REGENCIA,1752,25.714260,37.714260,12.0
7,Hugo Santos Zanatta,REGENCIA,160,51.428550,63.428550,12.0
26,Valquíria Alves,REGENCIA,149,1.714284,13.714284,12.0
20,Ruan Oliveira Olegário,REGENCIA,162,59.999940,73.999940,14.0
23,Sara Kowalski,REGENCIA,155,68.571390,84.571390,16.0
13,Marta Espíndola,REGENCIA,125,77.142840,95.142840,18.0
21,Rute Azevedo,REGENCIA,167,85.714260,105.714260,20.0


In [207]:
print(len(april_gov_agg_2))

29


In [211]:
april_tom_regency_2 = april_tom[(april_tom['Categoria'] == 'REGENCIA') & (april_tom['CodEscola'] == missing_school_codes[1])]

april_tom_regency_2['fator'] = april_tom_regency_2['Categoria']\
    .map({'REGENCIA': 4.5, 'NAO REGENCIA': 5})

april_tom_regency_2['CH_Semanal'] = april_tom_regency_2['Horas Mes'] / april_tom_regency_2['fator']

april_tom_regency_2.sort_values('CH_Semanal')

,Mês,Escola,CodEscola,Nome,CodFuncionario,SITUACAO,ADM,DESCRICAO_CARGO,DESCRICAO_VERBA,Horas Mes,Categoria,Dt demissão,fator,CH_Semanal
2058,Abril,ESCOLA G,ABC7,Pablo Lima Saraiva,992.0,Trabalhando,NaT,PROFESSOR(A),HORAS,18.0,REGENCIA,NaT,4.5,4.0
2046,Abril,ESCOLA G,ABC7,Kauê Serra,138.0,Trabalhando,NaT,PROFESSOR(A),HORAS,18.0,REGENCIA,NaT,4.5,4.0
2047,Abril,ESCOLA G,ABC7,NaN,NaN,Trabalhando,NaT,PROFESSOR(A),HORAS,18.9,REGENCIA,NaT,4.5,4.2
2044,Abril,ESCOLA G,ABC7,Isadora Donato,165.0,Trabalhando,NaT,PROFESSOR(A),HORAS,36.0,REGENCIA,NaT,4.5,8.0
2038,Abril,ESCOLA G,ABC7,Norberto Souza Zanetti,159.0,Trabalhando,NaT,PROFESSOR(A),HORAS,45.0,REGENCIA,NaT,4.5,10.0
2028,Abril,ESCOLA G,ABC7,Valquíria Alves,149.0,Trabalhando,NaT,PROFESSOR(A),HORAS,54.0,REGENCIA,NaT,4.5,12.0
2039,Abril,ESCOLA G,ABC7,Hugo Santos Zanatta,160.0,Trabalhando,NaT,PROFESSOR(A),HORAS,54.0,REGENCIA,NaT,4.5,12.0
2057,Abril,ESCOLA G,ABC7,Alícia Nakano,1752.0,Trabalhando,NaT,PROFESSOR(A),HORAS,54.0,REGENCIA,NaT,4.5,12.0
2041,Abril,ESCOLA G,ABC7,Ruan Oliveira Olegário,162.0,Trabalhando,NaT,PROFESSOR(A),HORAS,63.0,REGENCIA,NaT,4.5,14.0
2033,Abril,ESCOLA G,ABC7,Sara Kowalski,155.0,Trabalhando,NaT,PROFESSOR(A),HORAS,72.0,REGENCIA,NaT,4.5,16.0


In [212]:
print(len(april_tom_regency_2))

31


Outro fator importante pode ser descoberto aqui. Aparentemente o funcionário de código 992 (Pablo Lima Saraiva) nao consta nos dados da base de pagamentos do Governo. Apos a analise dos valores faltantes, trataremos as tabelas e verificaremos quais registros constam em ambas e quais possuem inconsistencias.

In [ ]:
missings_values_idxs.append(april_tom_regency_2[april_tom_regency_2['CodFuncionario'].isna()].index)


terceira escola

In [215]:
april_gov_per_missing_schools_3 = april_gov_per_missing_schools[(april_gov_filtered['CodEscola'] == missing_school_codes[2]) & (april_gov_per_missing_schools['Tipo'] == 'REGENCIA')]

april_gov_agg_3 = april_gov_per_missing_schools_3[['Nome Funcionario', 'Tipo', 'CHPeriodo', 'CHPeriodo_new', 'CHSemanal', 'CodFuncionario']].groupby(['Nome Funcionario', 'Tipo', 'CodFuncionario'])[['CHPeriodo', 'CHPeriodo_new', 'CHSemanal']].sum().reset_index()


april_gov_agg_3.sort_values('CHSemanal')

,Nome Funcionario,Tipo,CodFuncionario,CHPeriodo,CHPeriodo_new,CHSemanal
9,Maria Falcão,REGENCIA,611,59.99997,73.99997,14.0
12,Tatiana Siqueira,REGENCIA,588,64.28568,79.28568,15.0
13,Vera Lima,REGENCIA,593,68.57142,84.57142,16.0
7,Ivana Moraes,REGENCIA,592,72.85710,89.85710,17.0
14,Victor Santos Kawasaki,REGENCIA,597,102.85710,126.85710,24.0
3,Débora Nascimento,REGENCIA,595,119.99991,147.99991,28.0
1,Breno Pereira Medeiros,REGENCIA,589,124.28568,153.28568,29.0
8,Leandro Pinto,REGENCIA,598,124.28562,153.28562,29.0
2,Caio Oliveira Veloso,REGENCIA,585,128.57142,158.57142,30.0
6,Fábio Pereira Abreu,REGENCIA,587,128.57139,158.57139,30.0


In [216]:
april_tom_regency_3 = april_tom[(april_tom['Categoria'] == 'REGENCIA') & (april_tom['CodEscola'] == missing_school_codes[2])]

april_tom_regency_3['fator'] = april_tom_regency_3['Categoria']\
    .map({'REGENCIA': 4.5, 'NAO REGENCIA': 5})

april_tom_regency_3['CH_Semanal'] = april_tom_regency_3['Horas Mes'] / april_tom_regency_3['fator']

april_tom_regency_3.sort_values('CH_Semanal')

,Mês,Escola,CodEscola,Nome,CodFuncionario,SITUACAO,ADM,DESCRICAO_CARGO,DESCRICAO_VERBA,Horas Mes,Categoria,Dt demissão,fator,CH_Semanal
2617,Abril,ESCOLA W,ABC21,Tatiana Siqueira,588.0,Trabalhando,NaT,PROFESSOR(A),HORAS,5.00,REGENCIA,NaT,4.5,1.111111
2630,Abril,ESCOLA W,ABC21,Ivana Moraes,592.0,Trabalhando,NaT,PROFESSOR(A),HORAS,5.00,REGENCIA,NaT,4.5,1.111111
2628,Abril,ESCOLA W,ABC21,Maria Falcão,611.0,Trabalhando,NaT,PROFESSOR(A),HORAS,10.00,REGENCIA,NaT,4.5,2.222222
2632,Abril,ESCOLA W,ABC21,Vera Lima,593.0,Trabalhando,NaT,PROFESSOR(A),HORAS,10.00,REGENCIA,NaT,4.5,2.222222
2614,Abril,ESCOLA W,ABC21,Fábio Pereira Abreu,587.0,Trabalhando,NaT,PROFESSOR(A),HORAS,12.00,REGENCIA,NaT,4.5,2.666667
2640,Abril,ESCOLA W,ABC21,NaN,NaN,Trabalhando,NaT,PROFESSOR(A),HORAS,18.00,REGENCIA,NaT,4.5,4.000000
2610,Abril,ESCOLA W,ABC21,Caio Oliveira Veloso,585.0,Trabalhando,NaT,PROFESSOR(A),HORAS,20.00,REGENCIA,NaT,4.5,4.444444
2612,Abril,ESCOLA W,ABC21,Saulo Souza Cardoso,586.0,Trabalhando,NaT,PROFESSOR(A),HORAS,20.00,REGENCIA,NaT,4.5,4.444444
2634,Abril,ESCOLA W,ABC21,Débora Nascimento,595.0,Trabalhando,NaT,PROFESSOR(A),HORAS,20.00,REGENCIA,NaT,4.5,4.444444
2638,Abril,ESCOLA W,ABC21,Victor Santos Kawasaki,597.0,Trabalhando,NaT,PROFESSOR(A),HORAS,20.00,REGENCIA,NaT,4.5,4.444444
